### Example 1 - Race Conditions

Unsafe Counter:

In [ ]:
using System.Threading.Tasks;

int sharedCounter = 0;

// Simulate 10 tasks incrementing the counter 1000 times each
List<Task> tasks = new List<Task>();
for (int i = 0; i < 10; i++)
{
    tasks.Add(Task.Run(() => {
        for (int j = 0; j < 1000; j++)
        {
            // --- Potential Race Condition ---
            // 1. Read current value (e.g., 5)
            // 2. (Thread switch occurs) Another thread reads 5, increments to 6, writes 6.
            // 3. (Original thread resumes) Increments its old value (5) to 6.
            // 4. Writes 6 (overwriting the other thread's increment!)
            sharedCounter++;
            // --- End Race Condition ---
        }
    }));
}

await Task.WhenAll(tasks);

// Expected: 10 * 1000 = 10,000
// Actual: Likely *less* than 10,000 due to lost increments!
Console.WriteLine($"Final Counter (Unsafe): {sharedCounter}");


Final Counter (Unsafe): 7993


Safe Version using Locks:

In [2]:
using System.Threading.Tasks;

int sharedCounter = 0;
// Use a dedicated object for locking
private readonly object _counterLock = new object();

List<Task> tasks = new List<Task>();
for (int i = 0; i < 10; i++)
{
    tasks.Add(Task.Run(() => {
        for (int j = 0; j < 1000; j++)
        {
            // --- Critical Section Protected by Lock ---
            lock (_counterLock)
            {
                // Only one thread can be inside this block at a time
                sharedCounter++;
            }
        }
    }));
}

await Task.WhenAll(tasks);

// Expected: 10,000
// Actual: 10,000 (Correct!)
Console.WriteLine($"Final Counter (Safe with lock): {sharedCounter}");

Final Counter (Safe with lock): 10000


### Example 2 - Deadlocks

In [5]:
using System;
using System.Threading;
using System.Threading.Tasks;

// Assume _lockA and _lockB are defined elsewhere:
private static readonly object _lockA = new object();
private static readonly object _lockB = new object();

// Task 1: Acquires A then B
static void ProcessAB()
{
    Console.WriteLine("Task AB: Locking A...");
    lock (_lockA)
    {
        Console.WriteLine("Task AB: Locked A.");
        Thread.Sleep(100); // Simulate work holding A

        Console.WriteLine("Task AB: Trying to lock B...");
        lock (_lockB) // <<< Waits here if ProcessBA holds Lock B
        {
            Console.WriteLine("Task AB: Locked B.");
            // ... work ...
        }
        Console.WriteLine("Task AB: Released B.");
    }
    Console.WriteLine("Task AB: Released A.");
}

// Task 2: Acquires B then A (Inconsistent Order!)
static void ProcessBA()
{
    Console.WriteLine("Task BA: Locking B...");
    lock (_lockB)
    {
        Console.WriteLine("Task BA: Locked B.");
        Thread.Sleep(100); // Simulate work holding B

        Console.WriteLine("Task BA: Trying to lock A...");
        lock (_lockA) // <<< Waits here if ProcessAB holds Lock A - DEADLOCK!
        {
            Console.WriteLine("Task BA: Locked A.");
            // ... work ...
        }
        Console.WriteLine("Task BA: Released A.");
    }
    Console.WriteLine("Task BA: Released B.");
}

// To demonstrate deadlock potential:
Task task1 = Task.Run(ProcessAB);
Task task2 = Task.Run(ProcessBA);
await Task.WhenAll(task1, task2); // Likely hangs

Task BA: Locking B...
Task AB: Locking A...
Task AB: Locked A.
Task BA: Locked B.
Task BA: Trying to lock A...
Task AB: Trying to lock B...


Error: Command cancelled.

### Example 3 - Locking Lists

Unsafe Variation

In [22]:
using System.Collections.Generic;
using System.Threading.Tasks;

List<int> sharedList = new List<int>();
object listLock = new object(); // Dedicated lock object

// Task 1: Adds items
Task adderTask = Task.Run(() => {
    for (int i = 0; i < 100; i++)
    {
        sharedList.Add(i);
        Task.Delay(1).Wait();
    }
});

// Task 2: Reads and sums items (while Task 1 might be adding)
Task readerTask = Task.Run(() => {
    long sum = 0;
    try {
        Task.Delay(100).Wait(); // Small delay to increase chance of conflict
        foreach (int item in sharedList)
        {
            sum += item;
            Task.Delay(1).Wait(); // Small delay
        }
        Console.WriteLine($"Reader finished. Sum: {sum}"); // Sum likely incomplete/incorrect
    } catch (InvalidOperationException ex) {
        Console.WriteLine($"Reader failed: {ex.Message}"); // Very likely outcome!
    }
});

await Task.WhenAll(adderTask, readerTask);

Reader failed: Collection was modified; enumeration operation may not execute.


Safe Version:
 
Wrap every piece of code that reads or modifies the shared standard collection within a lock statement using the same dedicated lock object.

**Option 1:** Copy the list inside the lock for safe iteration outside


In [38]:
using System.Collections.Generic;
using System.Threading.Tasks;

List<int> sharedList = new List<int>();
object listLock = new object(); // Dedicated lock object

// Task 1: Adds items
Task adderTask = Task.Run(() => {
    for (int i = 0; i < 100; i++)
    {
        lock (listLock)
        {
            Task.Delay(1).Wait();
            sharedList.Add(i);
        }
    }
});

// Task 2: Reads and sums items (while Task 1 might be adding)
Task readerTask = Task.Run(() => {
    long sum = 0;
    try {
        Task.Delay(100).Wait(); // Small delay to increase chance of conflict
        List<int> listSnapshot;
        lock (listLock)
        {
            listSnapshot = sharedList.ToList(); // Creates a copy
        }
        foreach (int item in listSnapshot) { sum += item; }
        Console.WriteLine($"Reader finished. Sum: {sum}"); // Sum likely incomplete/incorrect
    } catch (InvalidOperationException ex) {
        Console.WriteLine($"Reader failed: {ex.Message}"); // Very likely outcome!
    }
});

await Task.WhenAll(adderTask, readerTask);

Reader finished. Sum: 28


**Option 2:** Iterate inside the lock


In [ ]:
using System.Collections.Generic;
using System.Threading.Tasks;

List<int> sharedList = new List<int>();
object listLock = new object(); // Dedicated lock object

// Task 1: Adds items
Task adderTask = Task.Run(() => {
    for (int i = 0; i < 100; i++)
    {
        lock (listLock)
        {
            Task.Delay(1).Wait();
            sharedList.Add(i);
        }
    }
});

// Task 2: Reads and sums items (while Task 1 might be adding)
Task readerTask = Task.Run(() => {
    long sum = 0;
    try {
        Task.Delay(100).Wait(); // Small delay to increase chance of conflict
        lock (listLock)
        {
            foreach (int item in sharedList) { sum += item; }
        }
        Console.WriteLine($"Reader finished. Sum: {sum}"); // Sum likely incomplete/incorrect
    } catch (InvalidOperationException ex) {
        Console.WriteLine($"Reader failed: {ex.Message}"); // Very likely outcome!
    }
});

await Task.WhenAll(adderTask, readerTask);

Reader finished. Sum: 91


### Example 4 - Concurrent Collections

In [48]:
using System.Collections.Concurrent;
using System.Threading.Tasks;

// Shared dictionary to store word counts
ConcurrentDictionary<string, int> wordCounts = new ConcurrentDictionary<string, int>();

// Simulate processing data sources in parallel
Task task1 = Task.Run(() => CountWords(new[] { "apple", "banana", "apple" }));
Task task2 = Task.Run(() => CountWords(new[] { "orange", "banana", "orange" }));
Task task3 = Task.Run(() => CountWords(new[] { "apple", "orange", "grape" }));

await Task.WhenAll(task1, task2, task3);

Console.WriteLine("Word Counts (ConcurrentDictionary):");
foreach (var kvp in wordCounts.OrderBy(kv => kv.Key))
{
    Console.Write($" | {kvp.Key}: {kvp.Value}");
}
Console.Write(" |");

// Helper method simulating processing words from a source
void CountWords(IEnumerable<string> words)
{
    foreach (string word in words)
    {
        // AddOrUpdate handles the logic:
        // - If new, add it with value 1.
        // - If exists, call the lambda to increment the current value.
        wordCounts.AddOrUpdate(word, 1, (key, currentCount) => currentCount + 1);
        Task.Delay(10).Wait();
    }
}


Word Counts (ConcurrentDictionary):
 | apple: 3 | banana: 2 | grape: 1 | orange: 3 |

## Example 5 Semasphores

In [62]:
// Imagine wanting to download 20 files, but only 3 downloads should run at once
// Starting all tasks immediately might cause too many concurrent downloads
List<Task> downloadTasks = new List<Task>();
for (int i = 1; i <= 20; i++)
{
    int fileNum = i;
    downloadTasks.Add(Task.Run(async () => { /* Download logic */ }));
}
// Problem: All 20 might start nearly simultaneously!
await Task.WhenAll(downloadTasks);


In [65]:
using System.Threading;
using System.Threading.Tasks;

// Allow max 3 concurrent operations
private static readonly SemaphoreSlim _throttler = new SemaphoreSlim(3, 3);

async Task DownloadFileAsync(int fileNum, CancellationToken token)
{
    Console.WriteLine($"[{fileNum}] Waiting for slot...");
    await _throttler.WaitAsync(token); // Wait for semaphore (pass token!)
    try
    {
        Console.WriteLine($"[{fileNum}] Entered slot. Downloading...");
        await Task.Delay(TimeSpan.FromSeconds(2), token); // Simulate download (pass token!)
        Console.WriteLine($"[{fileNum}] Finished download.");
    }
    finally
    {
        _throttler.Release(); // CRITICAL: Release the semaphore slot
        Console.WriteLine($"[{fileNum}] Slot released.");
    }
}

// --- How to use it ---
List<Task> downloadTasks = new List<Task>();
var cts = new CancellationTokenSource();
for (int i = 1; i <= 20; i++) { downloadTasks.Add(DownloadFileAsync(i, cts.Token)); }
await Task.WhenAll(downloadTasks); // Only 3 run concurrently inside DownloadFileAsync

[1] Waiting for slot...
[1] Entered slot. Downloading...
[2] Waiting for slot...
[2] Entered slot. Downloading...
[3] Waiting for slot...
[3] Entered slot. Downloading...
[4] Waiting for slot...
[5] Waiting for slot...
[6] Waiting for slot...
[7] Waiting for slot...
[8] Waiting for slot...
[9] Waiting for slot...
[10] Waiting for slot...
[11] Waiting for slot...
[12] Waiting for slot...
[13] Waiting for slot...
[14] Waiting for slot...
[15] Waiting for slot...
[16] Waiting for slot...
[17] Waiting for slot...
[18] Waiting for slot...
[19] Waiting for slot...
[20] Waiting for slot...
[3] Finished download.
[2] Finished download.
[2] Slot released.
[5] Entered slot. Downloading...
[1] Finished download.
[1] Slot released.
[6] Entered slot. Downloading...
[4] Entered slot. Downloading...
[3] Slot released.
[6] Finished download.
[4] Finished download.
[5] Finished download.
[6] Slot released.
[7] Entered slot. Downloading...
[8] Entered slot. Downloading...
[5] Slot released.
[9] Entered 

### Example 6 - Task Cancellation

In [66]:
using System.Threading;
using System.Threading.Tasks;

async Task StoppableProcessingLoop(CancellationToken token) 
{
    int itemsProcessed = 0;
    Console.WriteLine("Stoppable loop started.");
    try
    {
        while (true) // Loop indefinitely until cancelled
        {
            if (token.IsCancellationRequested)
            {
                Console.WriteLine("Exiting.");
                break; // Exit the loop cleanly
            }

            itemsProcessed++;
            Console.WriteLine($"Processing item {itemsProcessed}...");

            // Simulate work, passing the token to awaitable operations
            await Task.Delay(500, token);

            Console.WriteLine($"Finished item {itemsProcessed}.");
        }
    }
    catch (OperationCanceledException)
    {
        // This exception is expected if Task.Delay (or another cancellable method) is cancelled
        Console.WriteLine("Loop cancelled via OperationCanceledException.");
    }
    finally
    {
        Console.WriteLine($"Stoppable loop finished after processing {itemsProcessed} items.");
    }
}

// --- How to use it ---
var cts = new CancellationTokenSource();

Console.WriteLine("Starting stoppable processing loop...");
Task loopTask = StoppableProcessingLoop(cts.Token);

Console.WriteLine("Running for 3 seconds...");
await Task.Delay(3000);

Console.WriteLine("Requesting cancellation...");
cts.Cancel(); // Signal the loop to stop

await loopTask; // Wait for the loop task to actually finish cleaning up
Console.WriteLine("Loop task completed after cancellation.");
cts.Dispose();


Starting stoppable processing loop...
Stoppable loop started.
Processing item 1...
Running for 3 seconds...
Finished item 1.
Processing item 2...
Finished item 2.
Processing item 3...
Finished item 3.
Processing item 4...
Finished item 4.
Processing item 5...
Finished item 5.
Processing item 6...
Requesting cancellation...
Loop cancelled via OperationCanceledException.
Stoppable loop finished after processing 6 items.
Loop task completed after cancellation.


### Example 7 - Concurrent Tasks

In [ ]:
using System.Net.Http;
using System.Threading.Tasks;

// Assume httpClient is initialized
async Task<string> FetchUrlContentAsync(string url)
{
    Console.WriteLine($"Starting fetch: {url}");
    // Simulate network request
    await Task.Delay(TimeSpan.FromSeconds(new Random().Next(1, 4)));
    string content = $"Content from {url}";
    Console.WriteLine($"Finished fetch: {url}");
    return content;
}

// --- Using Task.WhenAll ---
string url1 = "https://example.com/page1";
string url2 = "https://example.com/page2";
string url3 = "https://example.com/page3";

Task<string> task1 = FetchUrlContentAsync(url1); // Start
Task<string> task2 = FetchUrlContentAsync(url2); // Start
Task<string> task3 = FetchUrlContentAsync(url3); // Start
Console.WriteLine("All fetch tasks started concurrently...");
string[] results = await Task.WhenAll(task1, task2, task3); // Wait for ALL
Console.WriteLine("All fetch tasks completed.");
// results[0] contains content from url1, results[1] from url2, etc.


In [67]:
using System.Threading;
using System.Threading.Tasks;

async Task<string> LongRunningOperation(CancellationToken token)
{
    Console.WriteLine("Long operation started...");
    await Task.Delay(5000, token); // Simulate 5 seconds work
    Console.WriteLine("Long operation finished.");
    return "Operation Result";
}

// --- Using Task.WhenAny for Timeout ---
var cts = new CancellationTokenSource();
Task<string> operationTask = LongRunningOperation(cts.Token);
Task timeoutTask = Task.Delay(TimeSpan.FromSeconds(3), cts.Token); // 3 second timeout

Console.WriteLine("Racing operation against 3-second timeout...");

// Wait for whichever task finishes first
Task completedTask = await Task.WhenAny(operationTask, timeoutTask);

if (completedTask == operationTask)
{
    // Operation finished before timeout
    Console.WriteLine("Operation completed successfully.");
    // We need to await the original task again to safely get the result or propagate exceptions
    string result = await operationTask;
    Console.WriteLine($"Result: {result}");
}
else
{
    // Timeout task finished first
    Console.WriteLine("Operation timed out!");
    // Important: Cancel the original operation if it timed out
    Console.WriteLine("Cancelling long operation...");
    cts.Cancel();
}
cts.Dispose();


Long operation started...
Racing operation against 3-second timeout...
Operation timed out!
Cancelling long operation...


### Example 8 - Periodic Timer

In [69]:
using System.Threading;
using System.Threading.Tasks;

// Assume token comes from a CancellationTokenSource (See previous slide!)
async Task RunPeriodicWork(CancellationToken token)
{
    // Create timer for a 5-second interval
    using var timer = new PeriodicTimer(TimeSpan.FromSeconds(2));

    try
    {
        // Loop continues as long as timer ticks and cancellation isn't requested
        while (await timer.WaitForNextTickAsync(token)) // Pass token here!
        {
            Console.WriteLine($"[{DateTime.Now:HH:mm:ss}] Doing periodic work...");
            // Perform your async or sync work here
            // await SomeAsyncWork(token);
        }
    }
    catch (OperationCanceledException)
    {
        Console.WriteLine("Periodic work cancelled."); // Expected when token is cancelled
    }
    finally
    {
        Console.WriteLine("Periodic work loop finished.");
    }
}

// To start:
var cts = new CancellationTokenSource();
Task workTask = RunPeriodicWork(cts.Token);
// To stop:
Task.Delay(10000).Wait(); 
cts.Cancel();

[13:43:25] Doing periodic work...
[13:43:27] Doing periodic work...
[13:43:29] Doing periodic work...
[13:43:31] Doing periodic work...
[13:43:33] Doing periodic work...
Periodic work cancelled.
Periodic work loop finished.


### Example 9 - Signaling

In [70]:
using System.Threading;
using System.Threading.Tasks;

// Starts in non-signaled state (Wait will block)
ManualResetEventSlim dataReadySignal = new ManualResetEventSlim(false);

string sharedData = null;

// Task 1: Waits for data, then processes it
Task consumer = Task.Run(() => {
    Console.WriteLine("Consumer: Waiting for data...");
    dataReadySignal.Wait(); // Blocks here until Set() is called
    Console.WriteLine("Consumer: Signal received! Processing data...");
    Console.WriteLine($"Consumer: Data is '{sharedData}'");
});

// Task 2: Prepares data, then signals
Task producer = Task.Run(async () => {
    Console.WriteLine("Producer: Preparing data...");
    await Task.Delay(2000); // Simulate work
    sharedData = "Hello from Producer!";
    Console.WriteLine("Producer: Data ready. Setting signal...");
    dataReadySignal.Set(); // Signal waiting threads
});

await Task.WhenAll(producer, consumer);
dataReadySignal.Dispose();
Console.WriteLine("Signaling example finished.");


Consumer: Waiting for data...
Producer: Preparing data...
Producer: Data ready. Setting signal...
Consumer: Signal received! Processing data...
Consumer: Data is 'Hello from Producer!'
Signaling example finished.


### Example 10 - Blocking Collections

In [50]:
using System.Collections.Concurrent;
using System.Threading;
using System.Threading.Tasks;

// Use default (ConcurrentQueue), optionally bounded (e.g., new BlockingCollection<string>(10))
BlockingCollection<string> messageQueue = new BlockingCollection<string>();

// Producer Task
Task producer = Task.Run(async () => {
    try {
        for (int i = 0; i < 5; i++) {
            string message = $"Message {i}";
            messageQueue.Add(message); // Blocks if bounded & full
            Console.WriteLine($"Producer added: {message}");
            await Task.Delay(1000); // Simulate work
        }
    } finally {
        // CRITICAL: Signal completion when done producing
        messageQueue.CompleteAdding();
        Console.WriteLine("Producer finished adding.");
    }
});

// Consumer Task
Task consumer = Task.Run(() => {
    try {
        // Loop efficiently waits, takes items, and exits when completed+empty
        foreach (string message in messageQueue.GetConsumingEnumerable()) // Blocks here if empty
        {
            Console.WriteLine($"\tConsumer processed: {message}");
            Task.Delay(2000).Wait(); // Simulate processing work
        }
         Console.WriteLine("\tConsumer finished processing (queue completed and empty).");
    } catch (OperationCanceledException) {
         Console.WriteLine("\tConsumer cancelled.");
    } catch (Exception ex) { Console.WriteLine($"\tConsumer Error: {ex.Message}"); }
});

await Task.WhenAll(producer, consumer);
messageQueue.Dispose(); // Dispose when done

Producer added: Message 0
	Consumer processed: Message 0
Producer added: Message 1
Producer added: Message 2
	Consumer processed: Message 1
Producer added: Message 3
	Consumer processed: Message 2
Producer added: Message 4
Producer finished adding.
	Consumer processed: Message 3
	Consumer processed: Message 4
	Consumer finished processing (queue completed and empty).


### Example 11 - Producer / Consumer Queue

In [118]:
public class PCQueue : IDisposable
{
  BlockingCollection<Task> _taskQ = new BlockingCollection<Task>();
  
  public PCQueue (int workerCount)
  {
    // Create and start a separate Task for each consumer:
    for (int i = 0; i < workerCount; i++)
    Task.Factory.StartNew (Consume);
  }
  
  public Task Enqueue (Action action, CancellationToken cancelToken = default (CancellationToken))
  {
    var task = new Task (action, cancelToken);
    _taskQ.Add (task);
    return task;
  }
  
  public Task<TResult> Enqueue<TResult> (Func<TResult> func, 
    CancellationToken cancelToken = default (CancellationToken))
  {
    var task = new Task<TResult> (func, cancelToken);
    _taskQ.Add (task);
    return task;
  }
  
  void Consume()
  {
    foreach (var task in _taskQ.GetConsumingEnumerable())
    try 
    {
      if (!task.IsCanceled) task.RunSynchronously();
    } 
    catch (InvalidOperationException) { }  // Race condition
  }
  
  public void Dispose() { _taskQ.CompleteAdding(); }
}


using (var pcQ = new PCQueue(1))
{
    Task t1 = pcQ.Enqueue (() => 
    {
        Task.Delay(5000).Wait(); 
        Console.WriteLine ("Too");
    });
    Task t2 = pcQ.Enqueue (() => 
    {
        Task.Delay(3000).Wait(); 
        Console.WriteLine ("Long!");
    });
    Task t3 = pcQ.Enqueue (() => 
    {
        Task.Delay(2000).Wait(); 
        Console.WriteLine ("Task!");
    });
    Task t4 = pcQ.Enqueue (() => 
    {
        Task.Delay(1000).Wait(); 
        Console.WriteLine ("Queue!");
    });
    Task.WaitAll (t1, t2, t3, t4); // Wait for all tasks to finish
    Task t5 = pcQ.Enqueue (() => 
    {
        Task.Delay(1000).Wait(); 
        Console.WriteLine ("Queue!");
    });
    await t5; // Wait for the last task to finish
}  



Too
Long!
Task!
Queue!
Queue!
